# Grammar 8B - SLR(1) Parser
**Student:** Riki Awal Syahputra (2802471404)  
**Attendance Number:** 22  
**Quiz:** COMP6062001 - Quiz 3 (Final Quiz)

---

## Grammar 8B
```
E → E + T
E → T
T → i
```

**Terminals:** `i`, `+`  
**Nonterminals:** `E`, `T`

## Productions

Augmented grammar with indexed productions:
```
0: E' → E
1: E → E + T
2: E → T
3: T → i
```

In [1]:
# Productions indexed from 0
# 0: E' → E
# 1: E → E + T
# 2: E → T
# 3: T → i

PRODUCTIONS = [
    ("E'", ['E']),      # 0
    ('E', ['E', '+', 'T']),  # 1
    ('E', ['T']),       # 2
    ('T', ['i']),       # 3
]

## ACTION Table

| State | i    | +    | $      |
|-------|------|------|--------|
| 0     | s3   | —    | —      |
| 1     | —    | s4   | accept |
| 2     | —    | r2   | r2     |
| 3     | —    | r3   | r3     |
| 4     | s6   | —    | —      |
| 5     | —    | r1   | r1     |
| 6     | —    | r3   | r3     |

In [2]:
# ACTION table: {state: {terminal: ('shift', next_state) | ('reduce', production_index) | ('accept',)}}
ACTION = {
    0: {'i': ('shift', 3)},
    1: {'+': ('shift', 4), '$': ('accept',)},
    2: {'+': ('reduce', 2), '$': ('reduce', 2)},
    3: {'+': ('reduce', 3), '$': ('reduce', 3)},
    4: {'i': ('shift', 6)},
    5: {'+': ('reduce', 1), '$': ('reduce', 1)},
    6: {'+': ('reduce', 3), '$': ('reduce', 3)},
}

## GOTO Table

| State | E | T |
|-------|---|---|
| 0     | 1 | 2 |
| 4     | — | 5 |

In [3]:
# GOTO table: {state: {nonterminal: next_state}}
GOTO = {
    0: {'E': 1, 'T': 2},
    4: {'T': 5},
}

## LR Parser Implementation

In [4]:
def parse_lr(tokens: list[str], verbose: bool = False) -> str:
    """
    LR table-driven parser.
    
    Args:
        tokens: List of token strings (should end with '$')
        verbose: If True, print parsing steps
    
    Returns:
        "ACCEPT" if input is valid, "ERROR" otherwise
    """
    # Ensure tokens end with $
    if not tokens or tokens[-1] != '$':
        tokens = tokens + ['$']
    
    stack = [0]  # Stack of states
    token_index = 0
    
    if verbose:
        print(f"{'Step':<6} {'Stack':<20} {'Input':<20} {'Action':<30}")
        print("-" * 76)
    
    step = 0
    while True:
        state = stack[-1]
        lookahead = tokens[token_index] if token_index < len(tokens) else '$'
        
        if verbose:
            stack_str = ' '.join(str(s) for s in stack)
            input_str = ' '.join(tokens[token_index:])
            print(f"{step:<6} {stack_str:<20} {input_str:<20}", end=" ")
        
        # Check ACTION table
        if state not in ACTION or lookahead not in ACTION[state]:
            if verbose:
                print("ERROR - no action")
            return "ERROR"
        
        action = ACTION[state][lookahead]
        
        if action[0] == 'shift':
            next_state = action[1]
            stack.append(next_state)
            token_index += 1
            if verbose:
                print(f"shift {next_state}")
        
        elif action[0] == 'reduce':
            prod_index = action[1]
            lhs, rhs = PRODUCTIONS[prod_index]
            
            # Pop |rhs| states from stack (unless rhs is epsilon)
            pop_count = len(rhs)
            if pop_count > 0:
                for _ in range(pop_count):
                    if len(stack) > 0:
                        stack.pop()
            
            # Get current state after popping
            if len(stack) == 0:
                if verbose:
                    print("ERROR - stack underflow")
                return "ERROR"
            
            current_state = stack[-1]
            
            # Use GOTO table
            if current_state not in GOTO or lhs not in GOTO[current_state]:
                if verbose:
                    print(f"ERROR - no GOTO[{current_state}][{lhs}]")
                return "ERROR"
            
            next_state = GOTO[current_state][lhs]
            stack.append(next_state)
            
            if verbose:
                rhs_str = ' '.join(rhs) if rhs else 'ε'
                print(f"reduce by {prod_index}: {lhs} → {rhs_str}")
        
        elif action[0] == 'accept':
            if verbose:
                print("ACCEPT")
            return "ACCEPT"
        
        else:
            if verbose:
                print("ERROR - unknown action")
            return "ERROR"
        
        step += 1
        
        # Safety check to prevent infinite loops
        if step > 1000:
            if verbose:
                print("ERROR - too many steps")
            return "ERROR"

## Test Cases

In [5]:
print("=" * 76)
print("Grammar 8B - SLR(1) Parser Test")
print("=" * 76)
print("\nGrammar:")
print("  E → E + T")
print("  E → T")
print("  T → i")
print("\n" + "=" * 76)

Grammar 8B - SLR(1) Parser Test

Grammar:
  E → E + T
  E → T
  T → i



### Test 1: Legitimate Input `i + i + i`
This is the **required legitimate input** from the quiz.

In [6]:
print("\nTest 1: Legitimate input 'i + i + i'")
print("-" * 76)
tokens1 = ['i', '+', 'i', '+', 'i', '$']
result1 = parse_lr(tokens1, verbose=True)
print(f"\nResult: {result1}")
print("Expected: ACCEPT")
print("✓ PASS" if result1 == "ACCEPT" else "✗ FAIL")


Test 1: Legitimate input 'i + i + i'
----------------------------------------------------------------------------
Step   Stack                Input                Action                        
----------------------------------------------------------------------------
0      0                    i + i + i $          shift 3
1      0 3                  + i + i $            reduce by 3: T → i
2      0 2                  + i + i $            reduce by 2: E → T
3      0 1                  + i + i $            shift 4
4      0 1 4                i + i $              shift 6
5      0 1 4 6              + i $                reduce by 3: T → i
6      0 1 4 5              + i $                reduce by 1: E → E + T
7      0 1                  + i $                shift 4
8      0 1 4                i $                  shift 6
9      0 1 4 6              $                    reduce by 3: T → i
10     0 1 4 5              $                    reduce by 1: E → E + T
11     0 1                 

### Test 2: Illegitimate Input `+ i i`
This is the **required illegitimate input** from the quiz.

In [7]:
print("\n" + "=" * 76)
print("\nTest 2: Illegitimate input '+ i i'")
print("-" * 76)
tokens2 = ['+', 'i', 'i', '$']
result2 = parse_lr(tokens2, verbose=True)
print(f"\nResult: {result2}")
print("Expected: ERROR")
print("✓ PASS" if result2 == "ERROR" else "✗ FAIL")



Test 2: Illegitimate input '+ i i'
----------------------------------------------------------------------------
Step   Stack                Input                Action                        
----------------------------------------------------------------------------
0      0                    + i i $              ERROR - no action

Result: ERROR
Expected: ERROR
✓ PASS


### Test 3: Single Identifier `i`
Additional test case - single identifier.

In [8]:
print("\n" + "=" * 76)
print("\nTest 3: Single identifier 'i'")
print("-" * 76)
tokens3 = ['i', '$']
result3 = parse_lr(tokens3, verbose=True)
print(f"\nResult: {result3}")
print("Expected: ACCEPT")
print("✓ PASS" if result3 == "ACCEPT" else "✗ FAIL")



Test 3: Single identifier 'i'
----------------------------------------------------------------------------
Step   Stack                Input                Action                        
----------------------------------------------------------------------------
0      0                    i $                  shift 3
1      0 3                  $                    reduce by 3: T → i
2      0 2                  $                    reduce by 2: E → T
3      0 1                  $                    ACCEPT

Result: ACCEPT
Expected: ACCEPT
✓ PASS


### Test 4: Invalid Input `i +`
Additional test case - incomplete expression.

In [9]:
print("\n" + "=" * 76)
print("\nTest 4: Invalid 'i +'")
print("-" * 76)
tokens4 = ['i', '+', '$']
result4 = parse_lr(tokens4, verbose=True)
print(f"\nResult: {result4}")
print("Expected: ERROR")
print("✓ PASS" if result4 == "ERROR" else "✗ FAIL")
print("\n" + "=" * 76)



Test 4: Invalid 'i +'
----------------------------------------------------------------------------
Step   Stack                Input                Action                        
----------------------------------------------------------------------------
0      0                    i + $                shift 3
1      0 3                  + $                  reduce by 3: T → i
2      0 2                  + $                  reduce by 2: E → T
3      0 1                  + $                  shift 4
4      0 1 4                $                    ERROR - no action

Result: ERROR
Expected: ERROR
✓ PASS

